### Get all chunks that need to be translated to English

In [35]:
import numpy as np
import openai
import os
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file
openai_api_key = os.environ.get("OPENAI_API_KEY")
client = openai.OpenAI()

In [37]:
from src.translation_src import translate_to_english, get_chunks_not_in_english

In [38]:
filtered_chunks = get_chunks_not_in_english(json_file_path= "../data/processed/defensewiki.ibj.org/unique_chunks.json")
print(f"Number of chunks to translate to english: {len(filtered_chunks)}")
links_list_to_extract = ['https://defensewiki.ibj.org/index.php?title=Burundi',
                         'https://defensewiki.ibj.org/index.php?title=Burundi/es',
                         'https://defensewiki.ibj.org/index.php?title=Burundi/fr']

filtered_chunks_2 = [c for c in filtered_chunks if c['metadata']['link'] in links_list_to_extract]
seen_countries = set([chunk['metadata']['country'] for chunk in filtered_chunks_2])

Number of chunks to translate to english: 3553


In [27]:
print(np.unique([(c['metadata']['title']) for c in filtered_chunks_2]))
print(len(filtered_chunks_2))

['Burundi-es' 'Burundi-fr']
19


### Translate one chunk

In [39]:
chunk = filtered_chunks_2[1]
md_text = chunk['content']
print(md_text)

Introduction

Le Burundi est un petit pays enclavé de la région des grands lacs d'Afrique qui lutte pour surmonter les conséquences d'une guerre civile qui aura durée plus de dix ans. Le nouveau gouvernement d'unité nationale dirigé par le Président Pierre Nkurunziza entreprend depuis 2005 la reconstruction de quasiment toutes les institutions du pays ainsi que le renforcement de l'état de droit et l'amélioration de la qualité de vie de ses citoyens. En avril 2009, le dernier groupe de rebelles du Burundi, les FNL (Forces de Libération Nationales) a renoncé à l'usage de la force et a été désarmé, créant ainsi une paix relativement stable dans le pays. Avec la large implication dans le gouvernement d'unité nationale des anciens groupes rebelles, la situation paraît encourageante.

Depuis la fin de la guerre civile de 12 ans, le Burundi a fait des progrès considérables en termes de normalisation sociale et d'ouverture de l'espace politique. Si le système judiciaire fait face à de nombreu

In [42]:
translated = translate_to_english(md_text, client)

In [43]:
print(translated)

# Introduction

Burundi is a small landlocked country in the Great Lakes region of Africa that struggles to overcome the consequences of a civil war that lasted more than ten years. The new national unity government led by President Pierre Nkurunziza has been undertaking the reconstruction of nearly all the country's institutions since 2005, as well as strengthening the rule of law and improving the quality of life for its citizens. In April 2009, the last group of Burundian rebels, the FNL (National Liberation Forces), renounced the use of force and was disarmed, thus creating a relatively stable peace in the country. With the significant involvement of former rebel groups in the national unity government, the situation appears encouraging.

Since the end of the 12-year civil war, Burundi has made considerable progress in terms of social normalization and the opening of political space. Although the judicial system faces many dysfunctions, representatives of law enforcement openly ack

In [44]:
translation_file = open("../data/interim/translation_file.txt", "a")
translation_file.write(f"Title: {chunk['title']}\n\n{chunk['metadata']}\n\nOriginal text:\n{md_text}\n\nTranslated text:\n{translated}\n\n\n\n")
translation_file.close()

### Translate several chunks using batches

In [45]:
from src.openai_batch_manager import upload_batch_file_to_openAI, submit_batch_job
from src.translation_src import create_batch_file_for_translation

In [ ]:
create_batch_file_for_translation(jsonl_output_file_path="../data/interim/batch_input_translation.jsonl", chunks=filtered_chunks_2)
file = upload_batch_file_to_openAI(batch_file_name="../data/interim/batch_input_translation.jsonl")
batch = submit_batch_job(file_id=file.id)
print("Batch job submitted:", batch.id)

In [47]:
batch_id="batch_6818c363af4c8190892dac4d68abbd84"

batch_6818c363af4c8190892dac4d68abbd84


In [48]:
batch = client.batches.retrieve(batch_id) #batch.id
print("Status:", batch.status)

Status: completed


In [49]:
result = client.batches.retrieve(batch_id="batch_6818c363af4c8190892dac4d68abbd84")

In [50]:
print(result)

Batch(id='batch_6818c363af4c8190892dac4d68abbd84', completion_window='24h', created_at=1746453347, endpoint='/v1/chat/completions', input_file_id='file-BHnLj7CqA8W1ML4h62Qn3z', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1746454765, error_file_id=None, errors=None, expired_at=None, expires_at=1746539747, failed_at=None, finalizing_at=1746454762, in_progress_at=1746453349, metadata=None, output_file_id='file-NfbVSfng5nVYKs4KMpzeZt', request_counts=BatchRequestCounts(completed=19, failed=0, total=19))


In [56]:
response = client.files.content(result.output_file_id)

with open("../data/interim/batch_results_translation.jsonl", "wb") as f:
    f.write(response.read())

In [ ]:
# TODO: create pipeline to do this for all our BURUNDI chunks
# Do we want the original content to be saved in the chunk?
# chunk['untranslated_content'] = chunk['content']
# chunk['content'] = translate_to_english(chunk['content'])
# TODO check this works and do this for all chunks
